In [2]:
# set module auto-reloaded 
%load_ext autoreload
%autoreload 2

In [3]:
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow import keras
import numpy as np
import dahuffman
import pickle
from Compressible_Huffman import Huffman
from getModel import get_simplemodel



In [6]:
# Load the CompressibleNN instance
import os

compNN_list = []
cnt = 0
directory = 'results_cifar_s'
decompressed_weights_list = []

while True:
    # Check if the file exists
    filename = f'{directory}/compressed_nn_{cnt}.pkl'
    if not os.path.exists(filename):
        break

    # Load the compressed model from the file
    with open(filename, 'rb') as compressed_nn:
        net_model = pickle.load(compressed_nn)
        codec = pickle.load(compressed_nn)
        compressed_model_weights = pickle.load(compressed_nn)

    # Create a new instance of CompressibleNN
    compNN = Huffman(net_model)
    compNN.codec = codec
    compNN_list.append(compNN)
    decompressed_weights = compNN.decompressNN(compressed_model_weights)
    decompressed_weights_list.append(decompressed_weights)
    
    # Increment the counter
    cnt += 1


AttributeError: 'str' object has no attribute 'get_weights'

In [ ]:
# Optional
# Compare the original weights with the decompressed weights
original_model_weights_list = []
for cnt, compNN in enumerate(compNN_list):
    # Check if the file exists
    filename = f'{directory}/original_model{cnt}_weights.pkl'
    if not os.path.exists(filename):
        break
    with open(filename, 'rb') as file:
        original_model_weights = pickle.load(file)
    original_model_weights_list.append(original_model_weights)
        
    differences = compNN.compare_weights(original_model_weights_list[cnt], decompressed_weights_list[cnt])

    print(f"Differences between original and decompressed weights in model{cnt}:")
    for i, diff in enumerate(differences):
        print(f"Layer {i+1}: Max Difference = {np.max(diff)}, Mean Difference = {np.mean(diff)}")
    print("-"*80)

In [ ]:
# set decompressed weights
for compNN, decompressed_weights in zip(compNN_list, decompressed_weights_list):
    compNN.set_weights(decompressed_weights)

In [5]:
from tensorflow.keras.datasets import mnist
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
image_size = x_train.shape[1] # 28
input_size = image_size * image_size

# Preprocess the data (reshape and normalize)
x_train = np.reshape(x_train, [-1, image_size, image_size, 3])  # Keep the shape (None, 28, 28, 1)
x_train = x_train.astype('float32') / 255
x_test = np.reshape(x_test, [-1, image_size, image_size, 3])  # Keep the shape (None, 28, 28, 1)
x_test = x_test.astype('float32') / 255

results = []

for cnt, compNN in enumerate(compNN_list): 
    # Make predictions using your model
    predictions = compNN.predict(x_test)

    # Calculate top-1 accuracy
    predicted_labels = tf.argmax(predictions, axis=1)
    top_1_accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted_labels, y_test), tf.float32)) * 100

    print(f"Model{cnt} Top-1 Accuracy: {top_1_accuracy:.3f}%")
    results.append(f"Model{cnt} Top-1 Accuracy: {top_1_accuracy:.3f}%")

ValueError: cannot reshape array of size 7840000 into shape (28,28,3)

In [90]:
# Define the full path to the log file
log_filename = os.path.join(directory, "accuracy_logs.txt")

# Save the results to the file
with open(log_filename, "w") as file:
    for result_entry in results:
        file.write(result_entry + "\n")